In [1]:
# Importing Required Libraries
import pandas as pd
import os
import mysql.connector

In [2]:
# Function to convert date format
def convert_date_format(date_series):
    return pd.to_datetime(date_series, format='%d/%m/%Y').dt.strftime('%Y/%m/%d')

In [3]:
# Function to process CSV files
def process_csv_files(file_path):
    all_dataframes = []  # Initialize a list to store all dataframes
    new_column_names = [  # Define new column names
        'Timestamp', 'ABS Pressure', 'Conductivity Linear', 'Conductivity NFL', 
        'Water Temp', 'Compensated Pressure', 'Barometric pressure', 
        'Casing temperature', 'EXT BATT', 'INT BATT', 
        'Min voltage', 'Max current', 'Signal', 'Signal strength', 'Access technology'
    ]

    # Iterate over all CSV files in the specified directory
    for filename in os.listdir(file_path):
        if filename.endswith('.csv'):
            file_path_full = os.path.join(file_path, filename)

            # Read CSV and convert to db_name
            db_name = pd.read_csv(file_path_full)

            # Drop rows 2 and 3 (index 0 and 1 since indexing starts at 0)
            db_name = db_name.drop([0, 1]).reset_index(drop=True)

            # Check if 'Date' column format is %d/%m/%Y
            try:
                test_date = pd.to_datetime(db_name['Date'], format='%d/%m/%Y', errors='raise')            
                db_name['Date'] = convert_date_format(db_name['Date'])
            except ValueError:            
                pass  

            # Merge 'Date' and 'Time' columns into a new column 'DateTime'
            db_name['DateTime'] = pd.to_datetime(db_name['Date'] + ' ' + db_name['Time'], format='%Y/%m/%d %H:%M:%S')

            # Drop the original 'Date' and 'Time' columns if no longer needed
            db_name = db_name.drop(columns=['Date', 'Time'])

            # Reorder columns to move 'DateTime' to the first position
            db_name = db_name[['DateTime'] + [col for col in db_name.columns if col != 'DateTime']]

            # Rename the first available columns up to 15
            num_columns_to_rename = min(len(db_name.columns), 15)
            db_name.columns.values[:num_columns_to_rename] = new_column_names[:num_columns_to_rename]

            # Store the processed DataFrame in the list
            all_dataframes.append(db_name)

    # Concatenate all DataFrames into a single DataFrame
    all_data = pd.concat(all_dataframes, ignore_index=True)

    # Drop columns with all NaN values
    all_data = all_data.dropna(axis=1, how='all')

    # Convert all columns to numeric types except the first column 'Timestamp'
    for col in all_data.columns[1:]:
        all_data[col] = pd.to_numeric(all_data[col], errors='coerce')

    return all_data

In [4]:
def insert_data_to_mysql(all_data, db_name):
    MYSQL_HOST = "localhost"
    MYSQL_USER = "mh30f"
    MYSQL_PASSWORD = "Moh@1977"
    MYSQL_DATABASE = "YDOCDatabase"

    connection = None
    cursor = None

    try:
        # Connect to MySQL without specifying a database first
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
        cursor = connection.cursor()

        # Create the database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_DATABASE}")

        # Select the database
        cursor.execute(f"USE {MYSQL_DATABASE}")

        # Create the YDOC table if it doesn't exist
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS YDOC (
            `Timestamp` DATETIME,
            `ABS_Pressure` FLOAT,
            `Conductivity_Linear` FLOAT,
            `Conductivity_NFL` FLOAT,
            `Water_Temp` FLOAT,
            `Compensated_Pressure` FLOAT,
            `Barometric_Pressure` FLOAT,
            `Casing_Temperature` FLOAT,
            `EXT_BATT` FLOAT DEFAULT NULL,
            `INT_BATT` FLOAT DEFAULT NULL,
            `Min_Voltage` FLOAT DEFAULT NULL,
            `Max_Current` FLOAT DEFAULT NULL,
            `Signal` FLOAT DEFAULT NULL,
            `Signal_Strength` FLOAT DEFAULT NULL,
            `Access_Technology` VARCHAR(50) DEFAULT NULL
        )
        """
        cursor.execute(create_table_query)

        # Replace spaces with underscores in column names
        all_data.columns = all_data.columns.str.replace(' ', '_')

        # Drop rows with NaN values
        all_data.dropna(inplace=True)

        # Dynamically generate column names and placeholders for SQL
        columns = all_data.columns
        placeholders = ', '.join(['%s'] * len(columns))  # Generates the required number of %s placeholders

        # SQL query with dynamic columns, using backticks for column names
        sql = f"INSERT INTO YDOC (`{'`, `'.join(columns)}`) VALUES ({placeholders})"

        # Now proceed with inserting data into the YDOC table
        for index, row in all_data.iterrows():
            cursor.execute(sql, tuple(row))

        # Commit the changes
        connection.commit()

    except mysql.connector.Error as e:
        print(f"An error occurred: {e}")

    finally:
        # Close the cursor and connection
        if cursor is not None:
            cursor.close()
        if connection is not None:
            connection.close()

In [5]:
def main():
    # Data path 
    file_path = r'C:\Users\mh30f\OneDrive\Documents\GitHub\YDOC2MySQL\Resources'
    
    # Process CSV files
    all_data = process_csv_files(file_path)

    # Insert data into MySQL
    insert_data_to_mysql(all_data, 'YDOCDatabase')

In [6]:
if __name__ == "__main__":
    main()